# google drive

utilities for google drive




## basic needs

authorization, listing, writing, copying, etc


### authorization
authorize google drive?


#### the code 



In [1]:
var fs = require('fs');
var path = require('path');
var {google} = require('googleapis');
var {GoogleAuth} = require('google-auth-library');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials;
if(fs.existsSync('./sheet to web-8ca5784e0b05.json')) {
    credentials = path.resolve('./sheet to web-8ca5784e0b05.json');
} else {
    credentials = path.join(PROFILE_PATH, '.credentials/sheet to web-8ca5784e0b05.json');
}

var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/drive'
];

function authorizeDrive() {
    return new GoogleAuth({
        keyFile: credentials,
        scopes: GOOGLE_AUTH_SCOPE
    }).getClient()
        .then(client => google.drive({version: 'v3', auth: client}))
}

module.exports = authorizeDrive;


[Function: authorizeDrive]

#### test list google drive?


In [ ]:
var importer = require('../Core');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

describe('list google drive files', () => {
    
    it('should list files', () => {
        return listDrive()
            .then(r => importer
                  .runAllPromises(r.filter(r => r.name.includes('Untitled'))
                                           .map(f => resolve => insertPermission(f.id, 'megamindbrian@gmail.com')
                                                .then(resolve))))
    })
})


### list google drive files?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeDrive = importer.import('authorize google drive');

function listDrive() {
    return authorizeDrive()
        .then(drive => util.promisify(drive.files.list.bind(drive))({}))
        .then(r => r.data.files || [])
}

module.exports = listDrive;


### insert google drive permissions?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeDrive = importer.import('authorize google drive');

function insertPermission(fileId, email) {
    return authorizeDrive()
        .then(drive => util.promisify(drive.permissions.create.bind(drive))({
            resource: {
                'type': 'user',
                'role': 'owner',
                'emailAddress': email
            },
            fileId: fileId,
            fields: 'id',
            transferOwnership: true
        }))
        .then(r => r.data.id)
}

module.exports = insertPermission;


### create a sheet in google drive?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeSheets = importer.import('authorize sheets api');
var insertPermission = importer.import('insert google drive permissions');

function createSheet(email) {
    var sheets;
    
    return authorizeSheets()
        .then(s => sheets = s)
        .then(() => util.promisify(sheets.spreadsheets.create.bind(sheets))())
        .then(r => insertPermission(r.data.spreadsheetId, 'megamindbrian@gmail.com'))
        .then(r => insertPermission(r.data.spreadsheetId, email))
}

module.exports = createSheet;


#### test google sheet create?


In [ ]:
var importer = require('../Core');
var createSheet = importer.import('create a sheet in google drive');

describe('create a new marketing sheet', () => {
    
    it('should create a sheet', () => {
        return createSheet('bjcullinan@gmail.com');
    })
})


### copy a file on google drive?


In [ ]:
var util = require('util');
var importer = require('../Core');
var authorizeDrive = importer.import('authorize google drive');

function copyFile(fileId, title) {
    return authorizeDrive()
        .then(drive => util.promisify(drive.files.copy.bind(drive))({
            fileId: fileId,
            resource: {name: title},
            convert: true
        }))
        .then(r => r.data.id)
}

module.exports = copyFile;


## advanced tools

comparing, cateloging, sharing, converting, etc



### compare a folder with local



#### the code 



In [24]:
var fs = require('fs');
var path = require('path');
var {GoogleAuth} = require('google-auth-library');
var importer = require('../Core')
var getRpcFromSpec = importer.import('get rpc from spec')

// TODO: pattern recognized! create a "google" object that does this for every service in the Google discovery list
var google = {drive: ({version, auth}) => getRpcFromSpec(require(path.join(__dirname, `../Resources/APIs/drive.${version}.json`)), auth)}

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var credentials;
if(fs.existsSync('./sheet to web-8ca5784e0b05.json')) {
    credentials = path.resolve('./sheet to web-8ca5784e0b05.json');
} else {
    credentials = path.join(PROFILE_PATH, '.credentials/sheet to web-8ca5784e0b05.json');
}

var GOOGLE_AUTH_SCOPE = [
    'https://www.googleapis.com/auth/drive'
];

async function authorizeDrive() {
    var auth = new GoogleAuth({
        keyFile: credentials,
        scopes: GOOGLE_AUTH_SCOPE
    })
    var client = await auth.getClient()
    return google.drive({version: 'v3', auth: client})
}



async function listDrive() {
    var drive = await authorizeDrive()
    var result = await drive.files.list({fields: 'files(id,name,modifiedTime,md5Checksum,mimeType,owners,originalFilename)'})
    return result.data
}

//module.exports = 

if(typeof $$ != 'undefined') {
    listDrive()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}



Error: Invalid field selection +originalFilename